<a href="https://colab.research.google.com/github/gsbiel/cv-homography/blob/main/custom_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Computer Vision Assignment - 2**

Students: 
- Gabriel Silva Gaspar; 
- Sara Sampaio Gomes do Nascimento.

In [2]:
def hello():
  return "hello from custom_cv!" 
